In [3]:
import cv2
import numpy as np
import mediapipe as mp

In [4]:
mp_face_detection = mp.solutions.face_detection
mp_fave_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils


In [5]:
mp_drawing_styles = mp.solutions.drawing_styles


In [6]:
face_mesh = mp_fave_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

In [7]:
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

In [8]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Ignoring empty camera frame.")
        continue
    
    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame and find face landmarks
    result = face_mesh.process(rgb_frame)
    
    # Draw the face landmarks
    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_fave_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())
    
    # Show the frame with landmarks
    cv2.imshow('MediaPipe FaceMesh', frame)
    
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\om\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [9]:
LEFT_EYE_LANDMARKS = [33 , 160 , 158 , 133 , 153 , 144]
RIGHT_EYE_LANDMARKS = [362 , 385 , 387 , 384 , 263 , 373]

In [10]:
def get_eye_center(landmarks , img_shape): 
    x_coords = [landmarks[i].x for i in range(len(landmarks))]
    y_coords = [landmarks[i].y for i in range(len(landmarks))]

    x_center = int(np.mean(x_coords) * img_shape[1])
    y_center = int(np.mean(y_coords) * img_shape[0])

    return (x_center , y_center)

In [13]:
cap = cv2.VideoCapture(0)

def get_eye_center(landmarks, img_shape):
    x_coords = [landmarks[i][0] for i in range(len(landmarks))]
    y_coords = [landmarks[i][1] for i in range(len(landmarks))]
    
    eye_center_x = int(np.mean(x_coords))
    eye_center_y = int(np.mean(y_coords))
    
    return (eye_center_x, eye_center_y)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Ignoring empty camera frame.")
        continue

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb_frame)
    
    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            landmarks = face_landmarks.landmark
            height, width, _ = frame.shape
            
            left_eye = [(int(landmarks[idx].x * width), int(landmarks[idx].y * height)) for idx in LEFT_EYE_LANDMARKS]
            right_eye = [(int(landmarks[idx].x * width), int(landmarks[idx].y * height)) for idx in RIGHT_EYE_LANDMARKS]
            
            left_eye_center = get_eye_center(left_eye, frame.shape)
            right_eye_center = get_eye_center(right_eye, frame.shape)
            
            cv2.circle(frame, left_eye_center, 3, (0, 255, 0), -1)
            cv2.circle(frame, right_eye_center, 3, (0, 255, 0), -1)
            
            # Calculate the gaze direction based on the eye centers
            if left_eye_center[0] < width // 3:
                if left_eye_center[1] < height // 3:
                    gaze_direction = "Looking Top-Left"
                elif left_eye_center[1] > 2 * height // 3:
                    gaze_direction = "Looking Bottom-Left"
                else:
                    gaze_direction = "Looking Left"
            elif left_eye_center[0] > 2 * width // 3:
                if left_eye_center[1] < height // 3:
                    gaze_direction = "Looking Top-Right"
                elif left_eye_center[1] > 2 * height // 3:
                    gaze_direction = "Looking Bottom-Right"
                else:
                    gaze_direction = "Looking Right"
            else:
                if left_eye_center[1] < height // 3:
                    gaze_direction = "Looking Up"
                elif left_eye_center[1] > 2 * height // 3:
                    gaze_direction = "Looking Down"
                else:
                    gaze_direction = "Looking Straight"

            # Detect unusual movements
            if abs(left_eye_center[0] - right_eye_center[0]) > width * 0.1 or abs(left_eye_center[1] - right_eye_center[1]) > height * 0.1:
                unusual_movement = "Unusual Eye Movement Detected"
            else:
                unusual_movement = "Normal Eye Movement"

            # Display the gaze direction and unusual movement status
            cv2.putText(frame, f'Gaze: {gaze_direction}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, unusual_movement, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
            cv2.putText(frame, f'Gaze: {gaze_direction}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv2.imshow('Gaze Tracking', frame)
    
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
